In [5]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from multiprocessing import Pool

In [7]:
'''
获取数据
'''
data=pd.read_excel('data.xlsx')
data1=data.copy()
data2=data1.loc[(data1['代码']!=1)&(data1['代码']!='')&(data1['代码']!='数据来源: Wind'),:]

In [6]:
industry = pd.read_csv('industry_data.csv',encoding='gbk')

In [7]:
def clean_data(industry):
    industry1 = industry[['日期','简称','前收盘价(元)', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)','成交量(股)', '成交金额(元)', '涨跌(元)', '涨跌幅(%)']]
    industry1['日期'] = pd.to_datetime(industry1['日期'], format="%Y/%m/%d")
    industry1 = industry1.sort_values(['日期', '简称']).reset_index().drop(columns = 'index')
    return industry1
industry1 = clean_data(industry)

/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
data2 = industry1.copy()
data2['y']=0
data2.loc[data2['涨跌幅(%)']>0,'y']=1


def create_y_leading(data):
    data['y_lead1']=data['y'].shift(-1)
    return data.fillna(-1)


def generate_lag_data(data1):
    diff_data=data1.groupby('简称')
    diff_data1=[i[1] for i in diff_data]
    pool=Pool(4)
    all_df=pool.map(create_y_leading,diff_data1) # 输入必须为list
    new_df=pd.concat(all_df)
    pool.close()
    return new_df

In [19]:
new_df = generate_lag_data(data2)

In [23]:
new_df = new_df[['日期', '简称', 'y_lead1']]

In [25]:
def split_name(x):
    return x.split('(')[0]

In [27]:
new_df['简称'] = new_df['简称'].apply(lambda x: split_name(x))

/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
new_df.to_excel('industry_train1.xlsx')

In [13]:
new_df= pd.read_csv('industry_train1.csv',encoding='gbk')

In [74]:
'''
数据处理1:
1.新增一列，上涨为1，下跌为0
2.构建划窗函数（3，5，7，9），构建每一个特征的移动平均值
3.将未来3日的涨跌标注在数据后
'''
data2['y']=0
data2.loc[data2['涨跌幅(%)']>0,'y']=1
columns=['收盘价(元)','成交量(股)','成交金额(元)','涨跌(元)','涨跌幅(%)']
code=data2['代码'].unique().tolist()

def create_y_leading(data):
    data['y_lead1']=data['y'].shift(-1)
    data['y_lead2']=data['y'].shift(-2)
    data['y_lead3']=data['y'].shift(-2)
    return data.fillna(-1)

def create_lag(data):
    windows=[3,5,7,9]
    for i in tqdm(windows):
        for j in columns:
            rolled=data[j].rolling(i)
            data['%s_lag%s_mean'%(j,i)]=rolled.mean()
            data['%s_lag%s_var'%(j,i)]=rolled.var()
            data['%s_lag%s_min'%(j,i)]=rolled.min()
            data['%s_lag%s_max'%(j,i)]=rolled.max()
    return data.fillna(-1)        
        
def generate_lag_data(data1):
    data=create_y_leading(data1)
    diff_data=data.groupby('代码')
    diff_data1=[i[1] for i in diff_data]
    pool=Pool(4)
    all_df=pool.map(create_lag,diff_data1)#输入必须为list
    new_df=pd.concat(all_df)
    pool.close()
    return new_df

In [75]:
new_df=generate_lag_data(data2)




  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 34.01it/s]





  0%|          | 0/4 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 27.55it/s]


 75%|███████▌  | 3/4 [00:00<00:00, 24.72it/s]





  0%|          | 0/4 [00:00<?, ?it/s]


 75%|███████▌  | 3/4 [00:00<00:00, 22.71it/s]


100%|██████████| 4/4 [00:00<00:00, 20.12it/s]


 50%|█████     | 2/4 [00:00<00:00, 18.85it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 18.06it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


 50%|█████     | 2/4 [00:00<00:00, 13.81it/s]


 75%|███████▌  | 3/4 [00:00<00:00, 14.40it/s]


 50%|█████     | 2/4 [00:00<00:00, 19.17it/s]


100%|██████████| 4/4 [00:00<00:00, 15.17it/s]


100%|██████████| 4/4 [00:00<00:00, 17.34it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 24.84it/s]


 75%|██

In [134]:
'''
数据处理2:
1.构造数据的特征值
2.构造全部构造傅立叶变换
'''
def get_fft_real(x):
    ''' 获取实部 '''
    zc=np.fft.fft(x)                                                                                     
    return np.real(zc)

def get_fft_imag(x):
    ''' 获取虚部 '''
    zc=np.fft.fft(x)
    return np.imag(zc)

def get_new_feature(data):
    data=data.reset_index().drop(columns='index')
    # 获取需要用的 columns
    columns=data.columns.tolist()
    col=['代码', '简称', '日期','y_lead1', 'y_lead2', 'y_lead3']
    cols=[i for i in columns if i not in col]
    num=data.shape[0]
    # 获取当前日期过去30天内的傅立叶特征
    for i in tqdm(cols):
        rolled=data[i].rolling(5)
        data[i+'_real_min']=rolled.apply(lambda x: get_fft_real(x).min())                                                      
        data[i+'_real_max']=rolled.apply(lambda x: get_fft_real(x).max())                                                        
        data[i+'_real_var']=rolled.apply(lambda x: get_fft_real(x).var())
        data[i+'_real_mean']=rolled.apply(lambda x: get_fft_real(x).mean())
        #data[i+'_real_min_max']= realFFT.max()- realFFT.min()    
        data[i+'_imag_min']=rolled.apply(lambda x: get_fft_imag(x).min()) 
        data[i+'_imag_max']=rolled.apply(lambda x: get_fft_imag(x).max()) 
        data[i+'_imag_var']=rolled.apply(lambda x: get_fft_imag(x).var()) 
        data[i+'_imag_mean']=rolled.apply(lambda x: get_fft_imag(x).mean()) 
        #data[i+'_imag_min_max']=imagFFT.max() -  imagFFT.min()
    return data.fillna(-1)


def generate_feature(data):
    # 将数据按照股票代码进行划分
    diff_data=data.groupby('代码')
    diff_data1=[i[1] for i in diff_data]
    pool=Pool(4)
    # 将分类后的数据放入 get_new_feature 中
    all_df=pool.map(get_new_feature,diff_data1)
    new_df=pd.concat(all_df)
    pool.close()
    return new_df.fillna(-1)  #空值默认填充为 -1



In [135]:
data_all=generate_feature(new_df)




  0%|          | 0/86 [00:00<?, ?it/s]


  0%|          | 0/86 [00:00<?, ?it/s]/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning



  0%|          | 0/86 [00:00<?, ?it/s]/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In t

/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, 

 65%|██████▌   | 56/86 [00:43<00:24,  1.21it/s]


 66%|██████▋   | 57/86 [00:44<00:24,  1.20it/s]


 66%|██████▋   | 57/86 [00:44<00:23,  1.22it/s]


 66%|██████▋   | 57/86 [00:44<00:23,  1.21it/s]


 66%|██████▋   | 57/86 [00:44<00:23,  1.24it/s]


 67%|██████▋   | 58/86 [00:45<00:22,  1.27it/s]


 67%|██████▋   | 58/86 [00:45<00:22,  1.24it/s]


 67%|██████▋   | 58/86 [00:45<00:22,  1.27it/s]


 67%|██████▋   | 58/86 [00:45<00:21,  1.27it/s]


 69%|██████▊   | 59/86 [00:45<00:20,  1.29it/s]


 69%|██████▊   | 59/86 [00:45<00:21,  1.27it/s]


 69%|██████▊   | 59/86 [00:46<00:21,  1.27it/s]


 69%|██████▊   | 59/86 [00:46<00:20,  1.30it/s]


 70%|██████▉   | 60/86 [00:46<00:20,  1.29it/s]


 70%|██████▉   | 60/86 [00:46<00:20,  1.28it/s]


 70%|██████▉   | 60/86 [00:46<00:19,  1.32it/s]


 70%|██████▉   | 60/86 [00:46<00:20,  1.27it/s]


 71%|███████   | 61/86 [00:47<00:19,  1.31it/s]


 71%|███████   | 61/86 [00:47<00:19,  1.30it/s]


 71%|███████   | 61/86 [00:47<00:18,  1.32it/s]




 45%|████▌     | 39/86 [00:28<00:38,  1.22it/s]


 45%|████▌     | 39/86 [00:28<00:38,  1.21it/s]


 45%|████▌     | 39/86 [00:28<00:38,  1.23it/s]


 45%|████▌     | 39/86 [00:28<00:37,  1.25it/s]


 47%|████▋     | 40/86 [00:29<00:35,  1.29it/s]


 47%|████▋     | 40/86 [00:29<00:35,  1.30it/s]


 47%|████▋     | 40/86 [00:29<00:35,  1.29it/s]


 47%|████▋     | 40/86 [00:29<00:35,  1.31it/s]


 48%|████▊     | 41/86 [00:30<00:33,  1.34it/s]


 48%|████▊     | 41/86 [00:30<00:33,  1.34it/s]


 48%|████▊     | 41/86 [00:30<00:33,  1.35it/s]


 48%|████▊     | 41/86 [00:30<00:33,  1.36it/s]


 49%|████▉     | 42/86 [00:30<00:32,  1.35it/s]


 49%|████▉     | 42/86 [00:30<00:32,  1.37it/s]


 49%|████▉     | 42/86 [00:30<00:32,  1.36it/s]


 49%|████▉     | 42/86 [00:30<00:31,  1.38it/s]


 50%|█████     | 43/86 [00:31<00:31,  1.37it/s]


 50%|█████     | 43/86 [00:31<00:31,  1.37it/s]


 50%|█████     | 43/86 [00:31<00:30,  1.40it/s]


 50%|█████     | 43/86 [00:31<00:30,  1.40it/s]




 23%|██▎       | 20/86 [00:10<00:37,  1.75it/s]


 24%|██▍       | 21/86 [00:11<00:36,  1.78it/s]


 26%|██▌       | 22/86 [00:11<00:34,  1.83it/s]


 26%|██▌       | 22/86 [00:11<00:33,  1.92it/s]


 24%|██▍       | 21/86 [00:11<00:35,  1.86it/s]


 26%|██▌       | 22/86 [00:11<00:33,  1.91it/s]


 27%|██▋       | 23/86 [00:12<00:32,  1.96it/s]


 26%|██▌       | 22/86 [00:11<00:32,  2.00it/s]


 27%|██▋       | 23/86 [00:12<00:31,  2.01it/s]


 27%|██▋       | 23/86 [00:12<00:31,  2.02it/s]


 28%|██▊       | 24/86 [00:12<00:30,  2.04it/s]


 28%|██▊       | 24/86 [00:12<00:30,  2.02it/s]


 27%|██▋       | 23/86 [00:12<00:31,  1.98it/s]


 28%|██▊       | 24/86 [00:12<00:31,  1.99it/s]


 29%|██▉       | 25/86 [00:13<00:31,  1.91it/s]


 29%|██▉       | 25/86 [00:13<00:31,  1.96it/s]


 28%|██▊       | 24/86 [00:12<00:31,  1.96it/s]


 29%|██▉       | 25/86 [00:13<00:31,  1.95it/s]


 30%|███       | 26/86 [00:13<00:30,  2.00it/s]


 30%|███       | 26/86 [00:13<00:30,  2.00it/s]




  3%|▎         | 3/86 [00:01<00:44,  1.85it/s]


  2%|▏         | 2/86 [00:01<00:46,  1.80it/s]


  5%|▍         | 4/86 [00:02<00:44,  1.85it/s]


  5%|▍         | 4/86 [00:02<00:42,  1.91it/s]


  3%|▎         | 3/86 [00:01<00:43,  1.90it/s]


  5%|▍         | 4/86 [00:02<00:43,  1.91it/s]


  6%|▌         | 5/86 [00:02<00:41,  1.97it/s]


  6%|▌         | 5/86 [00:02<00:39,  2.03it/s]


  6%|▌         | 5/86 [00:02<00:39,  2.03it/s]


  5%|▍         | 4/86 [00:02<00:40,  2.00it/s]


  7%|▋         | 6/86 [00:03<00:38,  2.05it/s]


  7%|▋         | 6/86 [00:02<00:37,  2.11it/s]


  7%|▋         | 6/86 [00:02<00:38,  2.10it/s]


  6%|▌         | 5/86 [00:02<00:38,  2.10it/s]


  8%|▊         | 7/86 [00:03<00:37,  2.11it/s]


  8%|▊         | 7/86 [00:03<00:36,  2.14it/s]


  7%|▋         | 6/86 [00:02<00:37,  2.14it/s]


  8%|▊         | 7/86 [00:03<00:37,  2.12it/s]


  9%|▉         | 8/86 [00:03<00:36,  2.16it/s]


  9%|▉         | 8/86 [00:03<00:36,  2.14it/s]


  8%|▊         | 7/8

 83%|████████▎ | 71/86 [00:38<00:10,  1.47it/s]


 85%|████████▍ | 73/86 [00:39<00:08,  1.52it/s]


 85%|████████▍ | 73/86 [00:39<00:08,  1.49it/s]


 85%|████████▍ | 73/86 [00:39<00:08,  1.54it/s]


 84%|████████▎ | 72/86 [00:39<00:09,  1.54it/s]


 86%|████████▌ | 74/86 [00:39<00:07,  1.54it/s]


 86%|████████▌ | 74/86 [00:40<00:08,  1.49it/s]


 86%|████████▌ | 74/86 [00:40<00:08,  1.48it/s]


 85%|████████▍ | 73/86 [00:39<00:08,  1.55it/s]


 87%|████████▋ | 75/86 [00:40<00:07,  1.55it/s]


 87%|████████▋ | 75/86 [00:40<00:07,  1.54it/s]


 87%|████████▋ | 75/86 [00:40<00:07,  1.48it/s]


 86%|████████▌ | 74/86 [00:40<00:07,  1.51it/s]


 88%|████████▊ | 76/86 [00:41<00:06,  1.50it/s]


 88%|████████▊ | 76/86 [00:41<00:06,  1.50it/s]


 88%|████████▊ | 76/86 [00:41<00:06,  1.48it/s]


 87%|████████▋ | 75/86 [00:41<00:07,  1.48it/s]


 90%|████████▉ | 77/86 [00:41<00:06,  1.46it/s]


 90%|████████▉ | 77/86 [00:42<00:06,  1.45it/s]


 90%|████████▉ | 77/86 [00:42<00:06,  1.46it/s]




 64%|██████▍   | 55/86 [00:24<00:15,  1.99it/s]


 62%|██████▏   | 53/86 [00:23<00:16,  2.01it/s]


 65%|██████▌   | 56/86 [00:25<00:14,  2.03it/s]


 64%|██████▍   | 55/86 [00:24<00:15,  2.01it/s]


 65%|██████▌   | 56/86 [00:25<00:14,  2.09it/s]


 63%|██████▎   | 54/86 [00:24<00:15,  2.13it/s]


 66%|██████▋   | 57/86 [00:25<00:13,  2.18it/s]


 65%|██████▌   | 56/86 [00:25<00:13,  2.17it/s]


 66%|██████▋   | 57/86 [00:25<00:13,  2.23it/s]


 64%|██████▍   | 55/86 [00:24<00:13,  2.24it/s]


 67%|██████▋   | 58/86 [00:25<00:12,  2.30it/s]


 66%|██████▋   | 57/86 [00:25<00:12,  2.32it/s]


 67%|██████▋   | 58/86 [00:25<00:12,  2.30it/s]


 65%|██████▌   | 56/86 [00:25<00:13,  2.29it/s]


 69%|██████▊   | 59/86 [00:26<00:11,  2.36it/s]


 67%|██████▋   | 58/86 [00:25<00:11,  2.39it/s]


 69%|██████▊   | 59/86 [00:26<00:11,  2.33it/s]


 66%|██████▋   | 57/86 [00:25<00:12,  2.36it/s]


 69%|██████▊   | 59/86 [00:26<00:11,  2.45it/s]


 70%|██████▉   | 60/86 [00:26<00:10,  2.39it/s]




 43%|████▎     | 37/86 [00:16<00:22,  2.15it/s]


 43%|████▎     | 37/86 [00:16<00:23,  2.10it/s]


 44%|████▍     | 38/86 [00:16<00:22,  2.11it/s]


 42%|████▏     | 36/86 [00:15<00:22,  2.20it/s]


 44%|████▍     | 38/86 [00:16<00:22,  2.15it/s]


 44%|████▍     | 38/86 [00:16<00:22,  2.11it/s]


 45%|████▌     | 39/86 [00:17<00:21,  2.14it/s]


 45%|████▌     | 39/86 [00:16<00:21,  2.17it/s]


 43%|████▎     | 37/86 [00:16<00:22,  2.16it/s]


 45%|████▌     | 39/86 [00:16<00:20,  2.24it/s]


 47%|████▋     | 40/86 [00:17<00:20,  2.30it/s]


 47%|████▋     | 40/86 [00:17<00:19,  2.37it/s]


 44%|████▍     | 38/86 [00:16<00:20,  2.33it/s]


 47%|████▋     | 40/86 [00:17<00:19,  2.42it/s]


 48%|████▊     | 41/86 [00:17<00:18,  2.44it/s]


 48%|████▊     | 41/86 [00:17<00:18,  2.45it/s]


 45%|████▌     | 39/86 [00:16<00:19,  2.45it/s]


 48%|████▊     | 41/86 [00:17<00:17,  2.52it/s]


 49%|████▉     | 42/86 [00:18<00:17,  2.53it/s]


 49%|████▉     | 42/86 [00:18<00:17,  2.55it/s]




 44%|████▍     | 38/86 [00:09<00:11,  4.28it/s]


 44%|████▍     | 38/86 [00:09<00:11,  4.29it/s]


 45%|████▌     | 39/86 [00:09<00:10,  4.39it/s]


 45%|████▌     | 39/86 [00:09<00:10,  4.45it/s]


 47%|████▋     | 40/86 [00:10<00:09,  4.63it/s]


 47%|████▋     | 40/86 [00:09<00:09,  4.67it/s]


 48%|████▊     | 41/86 [00:10<00:09,  4.83it/s]


 48%|████▊     | 41/86 [00:10<00:09,  4.84it/s]


 49%|████▉     | 42/86 [00:10<00:08,  4.91it/s]


 49%|████▉     | 42/86 [00:10<00:08,  4.93it/s]


 50%|█████     | 43/86 [00:10<00:08,  5.02it/s]


 50%|█████     | 43/86 [00:10<00:08,  5.01it/s]


 51%|█████     | 44/86 [00:10<00:08,  5.09it/s]


 51%|█████     | 44/86 [00:10<00:08,  5.06it/s]


 52%|█████▏    | 45/86 [00:10<00:08,  5.10it/s]


 52%|█████▏    | 45/86 [00:10<00:08,  5.09it/s]


 53%|█████▎    | 46/86 [00:11<00:07,  5.12it/s]


 53%|█████▎    | 46/86 [00:11<00:07,  5.10it/s]


 55%|█████▍    | 47/86 [00:11<00:07,  5.16it/s]


 55%|█████▍    | 47/86 [00:11<00:07,  5.06it/s]




In [143]:
print('数据处理完成,共获得 %s 条特征'%data_all.shape[1])

数据处理完成,共获得 780 条特征


In [145]:
data_all.to_excel('data_all.xlsx')

In [3]:
data3=pd.read_csv('data_all.csv')

In [4]:
data3.shape

(4560, 4624)

In [4]:
'''
划分数据集：按照时间划分
80%为训练集，20%为测试集
'''
from sklearn.model_selection import train_test_split
def split_data(data):
    columns=data.columns.tolist()
    target=['y_lead1','y_lead2','y_lead3']
    col=['代码','简称','日期','y_lead1','y_lead2','y_lead3']
    cols=[i for i in columns if i not in col]
    data_y=data[target]
    data_x=data[cols]
    num=data.shape[0]
    train_num=int(num*0.8)
    text_num=int(num*0.8)
    X_train, X_test, y_train, y_test=data_x.iloc[0:train_num,:],data_x.iloc[train_num:,:],data_y.iloc[0:train_num,:],data_y.iloc[train_num:,:]
    X_train, X_test, y_train, y_test=X_train.values, X_test.values, y_train.values, y_test.values
    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test=split_data(data3)

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/fancyjiang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [90]:
'''
搭建模型：x [None,774] y [None,3]
1. 获取数据，将数据进行打乱后切片
1. 构建模型：一层双向lstm，两层全连接层
2. 设置损失函数： 
3. 设置优化函数
4. 训练：
'''
def train_lstm(X_train, y_train):
    tf.reset_default_graph()
    
   
    x_shape = X_train.shape[1]
    X_train=X_train.tolist()
    y_train=y_train.tolist()
    
    with tf.variable_scope('prepared_data',reuse=True):
        x = tf.placeholder(dtype=tf.float32, shape=[None,774], name='x')
        y = tf.placeholder(dtype=tf.float32, shape=[None,3], name='y')

    with tf.variable_scope('create_model1',reuse=tf.AUTO_REUSE):
        # 1. 全连接层  [None,774]*[774,512]+[512]=[None,512]
        linear1_weights = tf.Variable(initial_value=tf.random_normal(shape=[x_shape,512]), name = 'linear1_weights')
        linear1_bias = tf.Variable(initial_value=tf.random_normal(shape=[512]), name = 'linear1_bias')
        linear_x1 = tf.add(tf.matmul(x,linear1_weights),linear1_bias, name = 'linear_x1')
        linear_x1 = tf.expand_dims(linear_x1,axis=1, name = 'linear_x11')
        # 2. 双向lstm in [None,512]  out [None,512]
        lstm_cell_fr = tf.nn.rnn_cell.BasicLSTMCell(num_units = 512 , forget_bias= 1.0 ,state_is_tuple= True)
        lstm_cell_ba = tf.nn.rnn_cell.BasicLSTMCell(num_units = 512 , forget_bias= 1.0 ,state_is_tuple= True)
        lstm_outputs , state = tf.nn.bidirectional_dynamic_rnn(lstm_cell_fr,lstm_cell_ba,linear_x1,dtype=np.float32)
        lstm_outputs = tf.concat(lstm_outputs,2, name = 'lstm_outputs')
        lstm_outputs = tf.squeeze(lstm_outputs, name = 'lstm_outputs1')
        # 3. 全连接层  in [None,512]  out [None,64]
        linear2_weights = tf.Variable(initial_value=tf.random_normal(shape=[1024,256]), name = 'linear2_weights')
        linear2_bias = tf.Variable(initial_value=tf.random_normal(shape=[256]), name = 'linear2_bias')
        linear_x2 = tf.add(tf.matmul(lstm_outputs ,linear2_weights),linear2_bias, name = 'linear_x2')
        linear3_weights = tf.Variable(initial_value=tf.random_normal(shape=[256,64]), name = 'linear3_weights')
        linear3_bias = tf.Variable(initial_value=tf.random_normal(shape=[64]), name = 'linear3_bias')
        linear_x3 = tf.add(tf.matmul(linear_x2 ,linear3_weights),linear3_bias, name = 'linear_x3')
        # 4. Dropout 0.3
        dropout_x = tf.nn.dropout(linear_x3, 0.3, noise_shape = None, name = 'dropout_x')
        # 5. 全连接层
        linear4_weights = tf.Variable(initial_value=tf.random_normal(shape=[64,3]), name = 'linear4_weights')
        linear4_bias = tf.Variable(initial_value=tf.random_normal(shape=[3]), name = 'linear4_bias')
        y_pred = tf.add(tf.matmul(dropout_x ,linear4_weights),linear4_bias, name ='y_pred')
        
    with tf.variable_scope('create_loss',reuse=True):
        loss = -tf.reduce_mean(y*tf.log(tf.clip_by_value(y_pred, 1e-10, 1.0)))
        
        
    with tf.variable_scope('create_optimizer',reuse=tf.AUTO_REUSE):
        optimizer =  tf.train.AdamOptimizer(0.1).minimize(loss)
    
    # 准确率计算
    bool_list = tf.equal(y, y_pred)
    accuracy = tf.reduce_mean(tf.cast(bool_list, tf.float32), name = 'accuracy')
    
    # 变量初始化
    init = tf.global_variables_initializer()
    
    #开启会话
    with tf.Session() as sess:
        sess.run(init)
        for i in range(1000):
            _, loss_value, accuracy_value = sess.run([optimizer, loss, accuracy], feed_dict={x : X_train, y : y_train})
            if i % 10 == 0:
                print("第%d次的损失为%f，准确率为%f" % (i+1, loss_value, accuracy_value))
            # print(sess.run(y_pred , feed_dict={x : X_train, y : y_train}))
        saver = tf.train.Saver()
        saver.save(sess, 'model')
    return None



In [91]:
sess = train_lstm(X_train, y_train)

W0405 11:10:44.039771 4548711872 deprecation.py:323] From /Users/fancyjiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


第1次的损失为6.554329，准确率为0.000000
第11次的损失为3.362885，准确率为0.000000
第21次的损失为2.947469，准确率为0.000000
第31次的损失为3.020002，准确率为0.000000
第41次的损失为3.092535，准确率为0.000000
第51次的损失为3.000220，准确率为0.000000
第61次的损失为3.185015，准确率为0.000000
第71次的损失为2.901312，准确率为0.000000
第81次的损失为1.859477，准确率为0.000000
第91次的损失为2.004542，准确率为0.000000
第101次的损失为2.096857，准确率为0.000000
第111次的损失为1.997948，准确率为0.000000
第121次的损失为1.991355，准确率为0.000000
第131次的损失为1.925416，准确率为0.000000
第141次的损失为2.050699，准确率为0.000000
第151次的损失为2.017730，准确率为0.000000
第161次的损失为2.004542，准确率为0.000000
第171次的损失为1.819913，准确率为0.000000
第181次的损失为2.116638，准确率为0.000000
第191次的损失为2.129826，准确率为0.000000
第201次的损失为2.057293，准确率为0.000000
第211次的损失为2.096857，准确率为0.000000
第221次的损失为2.057293，准确率为0.000000
第231次的损失为2.004542，准确率为0.000000
第241次的损失为2.083669，准确率为0.000000
第251次的损失为2.017730，准确率为0.000000


KeyboardInterrupt: 

In [100]:
def test_lstm(X_train, y_train):
   
    x_shape = X_train.shape[1]
    X_train=X_train.tolist()
    y_train=y_train.tolist()
    
    tf.reset_default_graph()
    
    with tf.variable_scope('prepared_data',reuse=True):
        x = tf.placeholder(dtype=tf.float32, shape=[None,774], name='x')
        y = tf.placeholder(dtype=tf.float32, shape=[None,3], name='y')

    with tf.variable_scope('create_model1',reuse=tf.AUTO_REUSE):
        # 1. 全连接层  [None,774]*[774,512]+[512]=[None,512]
        linear1_weights = tf.Variable(initial_value=tf.random_normal(shape=[x_shape,512]), name = 'linear1_weights')
        linear1_bias = tf.Variable(initial_value=tf.random_normal(shape=[512]), name = 'linear1_bias')
        linear_x1 = tf.add(tf.matmul(x,linear1_weights),linear1_bias)
        linear_x1 = tf.expand_dims(linear_x1,axis=1)
        # 2. 双向lstm in [None,512]  out [None,512]
        lstm_cell_fr = tf.nn.rnn_cell.BasicLSTMCell(num_units = 512 , forget_bias= 1.0 ,state_is_tuple= True)
        lstm_cell_ba = tf.nn.rnn_cell.BasicLSTMCell(num_units = 512 , forget_bias= 1.0 ,state_is_tuple= True)
        lstm_outputs , state = tf.nn.bidirectional_dynamic_rnn(lstm_cell_fr,lstm_cell_ba,linear_x1,dtype=np.float32)
        lstm_outputs1 = tf.concat(lstm_outputs,2)
        lstm_outputs = tf.squeeze(lstm_outputs1)
        # 3. 全连接层  in [None,512]  out [None,64]
        linear2_weights = tf.Variable(initial_value=tf.random_normal(shape=[1024,256]), name = 'linear2_weights')
        linear2_bias = tf.Variable(initial_value=tf.random_normal(shape=[256]), name = 'linear2_bias')
        linear_x2 = tf.add(tf.matmul(lstm_outputs ,linear2_weights),linear2_bias)
        linear3_weights = tf.Variable(initial_value=tf.random_normal(shape=[256,64]), name = 'linear3_weights')
        linear3_bias = tf.Variable(initial_value=tf.random_normal(shape=[64]), name = 'linear3_bias')
        linear_x3 = tf.add(tf.matmul(linear_x2 ,linear3_weights),linear3_bias)
        # 4. Dropout 0.3
        dropout_x = tf.nn.dropout(linear_x3, 0.3, noise_shape = None)
        # 5. 全连接层
        linear4_weights = tf.Variable(initial_value=tf.random_normal(shape=[64,3]), name = 'linear4_weights')
        linear4_bias = tf.Variable(initial_value=tf.random_normal(shape=[3]), name = 'linear4_bias')
        y_pred = tf.add(tf.matmul(dropout_x ,linear4_weights),linear4_bias, name ='y_pred')
        
    with tf.variable_scope('create_loss',reuse=True):
        loss_list = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_train)
        loss = tf.reduce_mean(loss_list)
        
    with tf.variable_scope('create_optimizer',reuse=tf.AUTO_REUSE):
        optimizer =  tf.train.AdamOptimizer(0.1).minimize(loss)
    
    # 准确率计算
    bool_list = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pred, axis=1))
    accuracy = tf.reduce_mean(tf.cast(bool_list, tf.float32), name = 'accuracy')
    
    saver = tf.train.Saver()
    # 变量初始化
    # init = tf.global_variables_initializer()
    
    
    #开启会话
    with tf.Session() as sess:
        saver.restore(sess, 'model')
        # print(sess.run(accuracy, feed_dict={x : X_test, y : y_test}))
        print(sess.run(lstm_outputs1, feed_dict={x : X_test, y : y_test}))
    return None

In [101]:
test_lstm(X_test, y_test)

[[[nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]]

 ...

 [[nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]]]


In [86]:
def test_lstm(X_test, y_test):
    X_test=X_test.tolist()
    y_test=y_test.tolist()
    
    with tf.Session() as sess:
        saver=tf.train.import_meta_graph('model.meta')   # end with .meta 文件
        graph=tf.get_default_graph()  
        tesor_name_list=[tensor.name for tensor in graph.as_graph_def().node]

        x = graph.get_tensor_by_name('prepared_data/x:0')
        y = graph.get_tensor_by_name('prepared_data/y:0')
        y_pred = graph.get_tensor_by_name('create_model1/y_pred:0')
        output1 =  graph.get_tensor_by_name('create_model1/lstm_outputs1:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        module_file = tf.train.latest_checkpoint('model')   # .meta 所在的文件夹名称
        saver.restore(sess, module_file)
        
        print(sess.run(output1 , feed_dict={x : X_test, y : y_test}))

    return None

In [87]:
 test_lstm(X_train, y_train)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
